## Training Machine Learning Algorithms using Nvidia Rapids cuML

In [2]:
!nvidia-smi

Sat May 22 20:35:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os, shutil

sys.path.append('/usr/local/lib/python3.7/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ["CONDA_PREFIX"] = "/usr/local"
for so in ['cudf', 'rmm', 'nccl', 'cuml', 'cugraph', 'xgboost', 'cuspatial']:
  fn = 'lib'+so+'.so'
  source_fn = '/usr/local/lib/'+fn
  dest_fn = '/usr/lib/'+fn
  if os.path.exists(source_fn):
    print(f'Copying {source_fn} to {dest_fn}')
    shutil.copyfile(source_fn, dest_fn)
# fix for BlazingSQL import issue
# ImportError: /usr/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.26' not found (required by /usr/local/lib/python3.7/site-packages/../../libblazingsql-engine.so)
if not os.path.exists('/usr/lib64'):
    os.makedirs('/usr/lib64')
for so_file in os.listdir('/usr/local/lib'):
  if 'libstdc' in so_file:
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib64/'+so_file)
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib/x86_64-linux-gnu/'+so_file)

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 238 (delta 36), reused 11 (delta 5), pack-reused 171
Receiving objects: 100% (238/238), 73.89 KiB | 1.27 MiB/s, done.
Resolving deltas: 100% (98/98), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT SCRIPT CHANGES: Colab has updated to Python 3.7, and now runs our STABLE and NIGHTLY versions (0.19 and 0.20)!  PLEASE update your older install script code as follows:
	!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.19

	import sys, os, shutil

	sys.path.append('/usr/local/lib/python3.7/site-packages/')
	os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
	os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
	os.environ['CONDA_PREFIX'] = '/usr/local'
	for so in ['cudf', 'rmm', 

In [8]:
import cudf
from cuml import make_regression, train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression

In [9]:
n_samples = 2**10 
n_features = 399
random_state = 23

In [10]:
## Lets generate some random regression data

%%time
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)

X = cudf.DataFrame(X)
y = cudf.DataFrame(y)[0]

X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)

CPU times: user 1.84 s, sys: 429 ms, total: 2.27 s
Wall time: 6.83 s


In [11]:
# Copy dataset from GPU memory to host memory.
# This is done to later compare CPU and GPU results.
X_train = X_cudf.to_pandas()
X_test = X_cudf_test.to_pandas()
y_train = y_cudf.to_pandas()
y_test = y_cudf_test.to_pandas()

In [13]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398
845,2.055562,-0.933736,0.497016,0.620149,-0.476680,-2.029445,0.123182,-0.641415,-0.553967,2.717702,0.860558,-0.313322,-1.051106,-1.380937,0.417878,-1.344837,-1.589228,2.148409,-1.095631,-0.140356,-0.079392,-0.659669,-0.360686,1.248658,-0.131434,-0.862661,0.005914,-0.601110,0.131767,0.742445,1.836354,-0.371129,-2.653161,-0.857406,-0.838788,-0.122874,0.531857,-0.696591,-0.073658,-0.223529,...,-1.144055,0.661332,0.625272,0.071821,-0.711666,0.647322,0.342235,-0.902038,-2.142352,1.303072,-1.025851,-0.403434,-2.296029,-1.235699,-0.070463,-0.645626,0.443654,0.981243,1.360011,-1.134194,0.520725,-0.123204,0.187861,-0.158220,0.078023,-0.184462,-1.095767,0.206128,-0.535497,-0.754671,-1.550962,2.538601,0.166509,1.863421,-0.352203,0.469129,-0.834900,-0.453884,1.550886,0.570708
480,-0.448590,1.736363,-0.136029,-0.022542,0.416618,-0.013258,-0.254864,0.336946,0.080474,-1.090549,-0.927134,1.667028,0.575787,-0.912464,0.348150,-0.145277,0.361353,0.701353,0.788017,-0.160181,1.499050,-2.972301,-0.755454,-1.087193,0.805894,-0.899627,-0.705622,1.018941,-0.489621,-0.082309,-0.371689,0.865737,-0.227379,1.083263,-0.304605,-0.760506,0.438162,-0.371897,1.617065,-0.390182,...,-1.885694,-1.417606,1.484554,1.966353,-0.387893,-1.300339,-0.103005,-0.046219,0.427647,-0.063938,-0.503205,1.253227,-1.085402,-0.131801,-2.345319,0.593710,1.220944,-0.603804,-0.205882,-1.609632,1.400760,-0.960835,0.122997,-1.220349,-0.838515,0.907545,0.869625,1.744500,-1.839492,-0.400698,-0.666103,0.192153,-0.032492,1.232339,0.966777,-0.450480,0.921368,-1.218761,0.573733,-0.126662
162,0.569151,-0.333358,-0.712302,0.676326,0.471869,0.722293,0.726290,-0.042724,0.303953,-0.012106,0.172228,0.166787,-2.062305,-1.726970,0.019084,-0.455605,-1.120517,0.702563,0.967574,-0.728839,-0.130154,1.152042,-0.912612,-0.569285,0.872799,-0.823302,-0.773043,-0.209472,0.795978,0.570768,-0.351556,-0.162365,-1.589545,0.698792,-0.082682,0.844782,-0.215189,0.202043,0.162839,-0.812618,...,1.263705,1.174916,0.619487,0.648082,-0.132168,-0.568357,1.199464,-1.199975,0.510640,-0.080806,0.733008,1.259133,-0.481431,-0.545911,1.100202,-0.375840,0.116208,0.693646,-0.953451,-1.012596,-1.226177,0.923681,0.338422,1.876497,-0.435721,1.367830,1.020849,1.914616,0.419945,1.050397,0.067205,0.939033,-0.855822,-0.434755,0.326109,-0.423419,-1.240627,0.734930,-0.179995,1.380971
49,0.236424,-0.600488,-1.037158,-0.069217,-0.675002,-0.724204,-1.039712,0.907647,1.059449,-0.061243,0.814336,0.304064,-0.159710,-0.810689,0.760082,-0.547184,-1.364508,0.139291,1.189483,0.199203,-1.382915,1.810462,-0.432328,-0.546016,0.611132,0.934809,0.545220,-0.206231,-1.417910,-1.023365,0.001771,0.141826,-0.490810,0.500067,0.164120,-0.129105,0.817416,-0.832125,-0.968688,-0.247358,...,0.144684,1.683117,-1.290303,2.003380,-0.926158,0.916690,0.528824,1.821189,0.634556,0.591865,0.035116,1.125213,0.415175,-0.733702,-0.775972,1.209593,0.380568,-0.351402,-2.038898,1.416754,1.702480,-0.588385,0.634842,1.032439,0.145111,-0.210908,-0.119754,-0.565198,0.312457,0.727650,1.027451,-0.505768,0.044367,0.559193,0.462736,1.526734,-0.822133,0.555926,-0.368712,-0.093048
251,-0.768838,-1.089635,-1.311010,-1.300188,0.857424,0.677873,0.522873,-0.150247,-0.648657,-0.198588,-1.276561,-0.635442,1.710104,-0.947975,-2.035677,1.178961,-1.545348,0.463087,1.776819,1.562142,-0.270043,0.384969,-1.102982,-0.395720,-0.495259,0.928657,1.002098,0.175542,-0.570441,1.842775,-2.024972,-0.763848,-0.446657,-0.664854,1.924501,-0.850967,-0.659818,-0.894578,-0.574243,-0.722197,...,0.095593,-1.604513,-0.174770,-1.318715,0.395460,-0.608084,-0.426315,0.320279,-1.090720,-0.218110,1.462598,0.591542,-0.765127,-0.473609,0.591032,0.522033,1.499483,0.386968,0.457194,0.129698,0.408810,2.392590,0.535800,0.206332,0.045365,0.475276,0.646595,-

In [12]:
y_train

845   -110.655334
480     90.493073
162   -129.152817
49     134.322357
251    -41.562408
          ...    
278   -106.089302
719     11.111393
999    -62.249809
758    -99.789841
240   -105.687286
Name: 0, Length: 820, dtype: float32

## Scikit-learn Model
## Fit, predict and evaluate

In [14]:
%%time
ols_sk = skLinearRegression(fit_intercept=True,
                            normalize=True,
                            n_jobs=-1)

ols_sk.fit(X_train, y_train)

CPU times: user 58.4 ms, sys: 40.6 ms, total: 99 ms
Wall time: 78 ms


In [15]:
%%time
predict_sk = ols_sk.predict(X_test)

CPU times: user 1.98 ms, sys: 0 ns, total: 1.98 ms
Wall time: 5 ms


In [17]:
%%time
r2_score_sk = r2_score(y_cudf_test, predict_sk)

CPU times: user 3.39 ms, sys: 1.9 ms, total: 5.29 ms
Wall time: 13.7 ms


## cuML Model
## Fit, predict and evaluate

In [18]:
%%time
ols_cuml = cuLinearRegression(fit_intercept=True,
                              normalize=True,
                              algorithm='eig')

ols_cuml.fit(X_cudf, y_cudf)

CPU times: user 49.6 ms, sys: 5.39 ms, total: 55 ms
Wall time: 90.9 ms


In [19]:
%%time
predict_cuml = ols_cuml.predict(X_cudf_test)

CPU times: user 30.9 ms, sys: 1.22 ms, total: 32.1 ms
Wall time: 33.1 ms


In [20]:
%%time
r2_score_cuml = r2_score(y_cudf_test, predict_cuml)

CPU times: user 1.59 ms, sys: 0 ns, total: 1.59 ms
Wall time: 1.6 ms


### Lets Compare Reuslts

In [21]:
print("R^2 score of SK Learn :  %s" % r2_score_sk)
print("R^2 score cuML: %s" % r2_score_cuml)

R^2 score of SK Learn :  1.0
R^2 score cuML: 1.0
